In [7]:
# from opensea import assets,bundles,common,events
# import json

from opensea_local import assets,bundles,common,events
import json

from PIL import Image
# load the image
import requests
from io import BytesIO

# JSON keys in API CALL

In [8]:
asset = assets.get_assets(asset_contract_address="0x06012c8cf97BEaD5deAe237070F9587f8E7A266d",limit=1,offset=1,verified_only=False)
print("First index asset keys")
asset[0].get_json().keys()

First index asset keys


dict_keys(['id', 'token_id', 'num_sales', 'background_color', 'image_url', 'image_preview_url', 'image_thumbnail_url', 'image_original_url', 'animation_url', 'animation_original_url', 'name', 'description', 'external_link', 'asset_contract', 'permalink', 'collection', 'decimals', 'token_metadata', 'owner', 'sell_orders', 'creator', 'traits', 'last_sale', 'top_bid', 'listing_date', 'is_presale', 'transfer_fee_payment_token', 'transfer_fee', 'related_assets', 'orders', 'auctions', 'supports_wyvern', 'top_ownerships', 'ownership', 'highest_buyer_commitment'])

In [9]:
# In depth look at json, keys such as collection has more subkeys
json_asset = json.dumps(asset[0].get_json(), indent = 4, sort_keys=True) 
print(json_asset)

{
    "animation_original_url": null,
    "animation_url": null,
    "asset_contract": {
        "address": "0x06012c8cf97bead5deae237070f9587f8e7a266d",
        "asset_contract_type": "non-fungible",
        "buyer_fee_basis_points": 0,
        "created_date": "2018-01-23T04:51:38.832339",
        "default_to_fiat": false,
        "description": "CryptoKitties is a game centered around breedable, collectible, and oh-so-adorable creatures we call CryptoKitties! Each cat is one-of-a-kind and 100% owned by you; it cannot be replicated, taken away, or destroyed.",
        "dev_buyer_fee_basis_points": 0,
        "dev_seller_fee_basis_points": 0,
        "external_link": "https://www.cryptokitties.co/",
        "image_url": "https://lh3.googleusercontent.com/C272ZRW1RGGef9vKMePFSCeKc1Lw6U40wl9ofNVxzUxFdj84hH9xJRQNf-7wgs7W8qw8RWe-1ybKp-VKuU5D-tg=s60",
        "name": "CryptoKitties",
        "nft_version": "1.0",
        "only_proxied_transfers": false,
        "opensea_buyer_fee_basis_poin

We can copy this and paste into http://jsonviewer.stack.hu/ to interactively see our variables we can use

If you take a look at opensea > models > asset.py 

``` Python
class Asset:
    def __init__(self, json_data):
        
        """
        Put useful information from json_data into their own variables under Asset class.

        :param json_data: json object returned by the opensea-api
        :type json_data: dict
        """
        # ASSET DETAILS
        self.name = json_data["name"]
        self.description = json_data["description"]
        self.token_id = json_data["token_id"]
        self.asset_url = json_data["permalink"]
        self.image_url = json_data["image_url"]
```

If we want to add say the created_date of the crypto we just need to add 
``` Python
self.created_date = json_data["created_date"]
```
and we will be able to call
``` Python
print(asset.created_date)
```

# A look into the Images from OpenSea

In [10]:
# This will return a list of assets which you can iterate and get the needed data
# Note we will ignore bundles (Collection of NFTs sold together)
from PIL import Image
# load the image
import requests
from io import BytesIO

asset_list = assets.get_assets(asset_contract_address="0x06012c8cf97BEaD5deAe237070F9587f8E7A266d",limit=3)

for i in range(len(asset_list)):
    asset = asset_list[i] # Get the first asset obejct from the list
    url = asset.image_url
    print(url)
    response = requests.get(url)
    try:
        img = Image.open(BytesIO(response.content))
        # summarize some details about the image
        print(img.format)
        print(img.mode)
        print(img.size)
        # show the image
        img.show()
    except Exception as inst:
        print(inst) 


asset: CryptoKitty #0, token_id: 0 has no last sale
https://storage.googleapis.com/ck-kitty-image/0x06012c8cf97bead5deae237070f9587f8e7a266d/0.svg
cannot identify image file <_io.BytesIO object at 0x0000019EAA1B8780>
https://lh3.googleusercontent.com/b9PIJNkqhgkRueWFXEaM_BDLoYzfIBWOt5O98Qq4fgWyDT9ClKxEXT9egD4I9cd1qtcCNrFV30mdu9HLYGfyUxSKQA
PNG
RGBA
(512, 512)
https://lh3.googleusercontent.com/ATyTdTkb984PCvY4e2Bd884RJZdPBbokfgg8sjmKe02kvzRKg_Rk1SmRTtqSuRp7brCcsw0G0_DVm8nxgPDDegI1
PNG
RGBA
(512, 512)


Note that we have images with different resolutions, this is something we will need to deal with, prehaps a simple rescale (svd?)

# A Look into downloading Videos

In [15]:
from PIL import Image
# load the image
import requests
from io import BytesIO
import mimetypes
import urllib.request
import time

output_dir="data/raw/media"
for i in range(0,300,30):
    print(i)
    asset_list = assets.get_assets(asset_contract_address="0xa98771a46dcb34b34cdad5355718f8a97c8e603e",limit=30,offset=i,order_direction="desc")
    for asset in asset_list:
        try:
            url = asset.animation_url
            print(url)
            response = requests.get(url)
            content_type = response.headers['content-type']
            extension = mimetypes.guess_extension(content_type)
            try:    
                with open("{}/{}_{}{}".format(output_dir,asset.collection_slug,asset.token_id,extension), 'wb') as f:
                    f.write(response.content)
            except:
                pass
            try:
                urllib.request.urlretrieve(url, "{}/{}_{}{}".format(output_dir,asset.collection_slug,asset.token_id,extension)) 
            except:
                pass
        except:
            try:
                url = asset.animation_original_url
                print(url)
                response = requests.get(url)
                content_type = response.headers['content-type']
                extension = mimetypes.guess_extension(content_type)
                try:    
                    with open("{}/{}_{}{}".format(output_dir,asset.collection_slug,asset.token_id,extension), 'wb') as f:
                        f.write(response.content)
                except:
                    pass
                try:
                    urllib.request.urlretrieve(url, "{}/{}_{}{}".format(output_dir,asset.collection_slug,asset.token_id,extension)) 
                except:
                    pass
            except Exception as animation_inst:
                print(animation_inst) 
    time.sleep(5)

0
asset: EulerBeats Enigma LP 24 Print, token_id: 4432574481153 has no last sale
asset: EulerBeats Enigma LP 27 Print, token_id: 4398080131841 has no last sale
asset: EulerBeats Enigma LP 27, token_id: 4398080131840 has no last sale
asset: EulerBeats Enigma LP 20 Print, token_id: 1348636967681 has no last sale
asset: EulerBeats Enigma LP 06 Print, token_id: 1340164079617 has no last sale
asset: EulerBeats Enigma LP 19 Print, token_id: 1331557565185 has no last sale
asset: EulerBeats Enigma LP 05 Print, token_id: 1331490521857 has no last sale
asset: EulerBeats Enigma LP 18 Print, token_id: 1297180984065 has no last sale
asset: EulerBeats Enigma LP 26 Print, token_id: 1280018153729 has no last sale
asset: EulerBeats Enigma LP 26, token_id: 1280018153728 has no last sale
https://storage.opensea.io/files/fcad3ebcdb925f7b1020899fa9638474.mp4
https://storage.opensea.io/files/fcad3ebcdb925f7b1020899fa9638474.mp4
https://storage.opensea.io/files/dada705aec24c043ea4347d65a00db05.mp4
https://st

In [13]:
# This will return a list of assets which you can iterate and get the needed data
# Note we will ignore bundles (Collection of NFTs sold together)

asset_list = assets.get_assets(asset_contract_address="0x06012c8cf97BEaD5deAe237070F9587f8E7A266d",limit=5,offset=1,verified_only=True)

for i in range(len(asset_list)):
    asset = asset_list[i] # Get the first asset obejct from the list

    # print(asset.owner.address)
    # print(asset.owner.profile_img_url)
    # print(asset.owner.username)
    # print(asset.owner.config)

    print(asset.name)
    print(asset.token_id)
    print(asset.description)
    print(asset.asset_url)
    print(asset.image_url)
    print(asset.get_floor_price()) # Floor price of the collection

Genesis
1
Greetings, human. I am Genesis. The dogs know me as alpha; the cats know me as omega. To your kind, I am a riddle wrapped in an enigma, first found by a user in Mystery, Alaska. I looked into the void and the void looked back. Then I lost interest. I can’t wait to be your new owner!
https://opensea.io/assets/0x06012c8cf97bead5deae237070f9587f8e7a266d/1
https://lh3.googleusercontent.com/b9PIJNkqhgkRueWFXEaM_BDLoYzfIBWOt5O98Qq4fgWyDT9ClKxEXT9egD4I9cd1qtcCNrFV30mdu9HLYGfyUxSKQA
0
Riker | Founder Cat #2
2
Hi-ya! I'm Riker | Founder Cat #2. I'm often referred to as the Lisa Simpson of the group. Sometimes I daydream of a life full of picking flowers, ice cream, and prank-calling celebrities. Can you make my annoying dreams come true?
https://opensea.io/assets/0x06012c8cf97bead5deae237070f9587f8e7a266d/2
https://lh3.googleusercontent.com/ATyTdTkb984PCvY4e2Bd884RJZdPBbokfgg8sjmKe02kvzRKg_Rk1SmRTtqSuRp7brCcsw0G0_DVm8nxgPDDegI1
0
Founder Cat #3
3
What's up! I'm Founder Cat #3. I'm a p

# A look into querying 1 NFT collection

In [14]:

# This will return a list of assets which you can iterate and get the needed data
# Note we will ignore bundles (Collection of NFTs sold together)

# Order by sale_price to get order by last total_price

asset_list = assets.get_assets(collection="cryptokitties",limit=1, verified_only=True, token_ids=896775)

for i in range(len(asset_list)):
    asset = asset_list[i] # Get the first asset obejct from the list
    print("i:", i)
    # print(asset.owner.address)    

    print(asset.name)
    print(asset.description)
    print(asset.asset_url)
    # print(asset.image_url)
    print("price floor", asset.get_floor_price()) # Floor price of the collection

    


TypeError: get_assets() got an unexpected keyword argument 'collection'

In [ ]:
# In depth look at json, keys such as collection has more subkeys
json_asset = json.dumps(asset_list[0].get_json(), indent = 4, sort_keys=True) 
print(json_asset)

# Looking at NFT EVENT transactions

In [ ]:
asset_list = events.get_events(event_type='successful',asset_contract_address="0x06012c8cf97BEaD5deAe237070F9587f8E7A266d", token_id=896775)
asset_list


In [ ]:
asset_list["asset_events"][0]

# Running through to see a how many kitties are currently on sale, or have a last_price

In [ ]:
counter = 0 
index = []
import time
for i in range(1560, 10000):
    try:
        asset_list = assets.get_assets(asset_contract_address="0xb47e3cd837dDF8e4c57F05d70Ab865de6e193BBB",limit=30,verified_only=True,token_ids=list(range((i*30)+1, (i*30)+31)))
        print(i)
        index.append(i)
    except Exception as inst:
        if inst.args[0] == "'NoneType' object is not subscriptable":
            continue
        else:
            print(i)
            index.append(i)

In [ ]:
index

In [ ]:
counter = 0 
cats = []
import time
for i in range(1500, 10000):
    try:
        #print(list(range((i*30)+1, (i*30)+31)))
        asset_list = assets.get_assets(asset_contract_address="0xb47e3cd837dDF8e4c57F05d70Ab865de6e193BBB",limit=30,verified_only=True)
        print("i:", i)
        print(asset_list)
        try:
            if asset_list.status_code == 429:
                time.sleep(20)
        except:
            pass
        for j in range(len(asset_list)):
            asset = asset_list[j] # Get the first asset obejct from the list
            counter += 1
            cats.append((counter,str(asset.name), str(int(asset.last_sale)/1000000000000000000),asset.token_id))
    except Exception as inst:
        print(inst)
    time.sleep(5)


In [ ]:
cats

In [ ]:
asset_list = assets.get_assets(collection="cryptokitties",limit=30, verified_only=True, order_by="sale_price")
for i in range(len(asset_list)):
    asset = asset_list[i] # Get the first asset obejct from the list
    print("i:", i)
    # print(asset.owner.address)    

    print(asset.name)
    print(asset.description)
    print(asset.asset_url)
    # print(asset.image_url)
    print(asset.token_id)
    print("price floor", asset.get_floor_price()) # Floor price of the collection

In [ ]:
import urllib.request
import os
import mimetypes

url = "https://storage.opensea.io/files/bb9dce1d20053950ca2449cd0733d225.mp3"
response = requests.get(url)
content_type = response.headers['content-type']
extension = mimetypes.guess_extension(content_type)
print(extension)
os.makedirs("data/raw/contents",exist_ok=True)
try:    
    with open("data/raw/contents/{}_{}_{}".format("1","2",extension), 'wb') as f:
        f.write(response.content)
except:
    pass
try:
    urllib.request.urlretrieve(url, "data/raw/contents/{}_{}_{}".format("1","2",extension)) 
except:
    pass
    

